Ноутбук для расписи всего алгоритма обработки данных

Импорт библиотек

In [1]:
import pandas as pd
import numpy as np
import re

# import seaborn as sns
# import matplotlib.pyplot as plt
# import missingno as msn

Загружаем данные

In [2]:
df = pd.read_csv('../train.csv')

C:\Users\egorb\AppData\Local\Temp\ipykernel_16036\1437252748.py:1: DtypeWarning: Columns (51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,131,132,133,135,136,139,140,141,142,143,144,145,146,147,148,149,151,152,155,156,157,158,159,160,161,163,164,165,167,168,169,171,172,173,174,175,176,177,178,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,223,224,227,239,240,241,242,243,244,245,246,459,460,461,467,468,469,515,516,523,524,531,532,539,540,547,548,555,556,563,564,565,567,571,572,573,575,579,580,581,582,583,587,588,595,596,603,604,611,612,613,615,795,796,797,799,1031,1032,1033,1034,1035,1036,1037,1039,1040,1041,1042,1043,1044,1045,1047,1048,1049,1050,1051,1052,1055,1056,1057,1058,1059,1060,1063,1064,1065,1067,1068,1069,1072,1073,1074,1075,1076,1077,1078,1172,1173,1174,1176,1177,1178,1179,1180,1181,1182,1275,1276,1277,1278,1279,1280,1281,1282,1447,1448,1449,145

Убираем дупликаты и полностью пустые колонки

In [ ]:
df.dropna(how='all', axis=1, inplace=True)
df.drop_duplicates()

Переводим все возможные колонки в INT64

In [4]:
float_columns = df.select_dtypes(include=np.number).columns
for col in float_columns:
    temp = df[col].fillna(0)
    if temp.apply(lambda x: x == int(x)).all():
        df[col] = df[col].astype("Int64")
df = df.loc[:, (df != 0).any(axis=0)]

Разбиваем датафрейм по типам данных

In [5]:
df_float = df.select_dtypes(include=["float64"])
df_int = df.select_dtypes(include=["int64"])
df_object = df.select_dtypes(include=["object"])

Merge'им каждый датафрейм

In [6]:
df_object = df_object.astype(str)
df_object.replace('0', '', inplace=True)
df_object.replace('nan', np.nan, inplace=True)

def merge_columns_with_common_values(df):
    global count
    count = 0
    columns = df.columns.tolist()
    merged_columns = []
    i = 0
    while i < len(columns) - 1:
        current_col = columns[i]
        next_col = columns[i + 1]
        
        common_values = list(set(df[current_col].unique()).intersection(set(df[next_col].unique())))
        x = np.nan
        if(x in common_values): common_values.remove(x)
        if common_values:
            count += 1
            merged_name = f"{current_col}"
            df[merged_name] = df[current_col].combine_first(df[next_col])
            merged_columns.append(merged_name)
            i += 2  
        else:
            merged_columns.append(current_col)
            i += 1
    
    if i == len(columns) - 1:
        merged_columns.append(columns[-1])
    
    df = df[merged_columns]
    return df

df_object = merge_columns_with_common_values(df_object)
i = 1
while count != 0:
    i += 1
    df_object = merge_columns_with_common_values(df_object)
    print(f"Iteration {i}: {len(df_object.columns)} columns left")

if count == 0:
    print("Nothing changed!")


Iteration 2: 89 columns left
Iteration 3: 46 columns left
Iteration 4: 39 columns left
Iteration 5: 37 columns left
Iteration 6: 37 columns left
Nothing changed!


In [ ]:
def merge_columns_with_common_values(df):
    base_toggle = True
    count = 0
    unique_columns = []
    temp_col = "col4"
    
    df.replace(["nan", np.float64(0.0), np.float64(1.0)], np.nan, inplace=True)
    
    column_groups = {}

    for col in df.columns:
        if col in unique_columns:  continue
        
        if col != df.columns[1]: base_toggle = False

        column_groups[col] = [col]
        
        for next_col in df.columns:
            if col == next_col or next_col in unique_columns: continue
            
            common_values = list(set(df[col].unique()).intersection(set(df[next_col].unique())))
            if np.nan in common_values: common_values.remove(np.nan)
            
            match = re.match(r"([a-zA-Z]+)(\d{1,4})", col)
            match1 = re.match(r"([a-zA-Z]+)(\d{1,4})", next_col)
            def calc_jump():
                if base_toggle == True: return False
                else:
                    match = re.match(r"([a-zA-Z]+)(\d{1,4})", temp_col)
                    is_jump = ((int(match1.group(2)) - int(match.group(2))) > 5)
                    return is_jump

            if len(common_values) >= 1:
                if calc_jump():
                    temp_col = next_col
                    break
                count += 1
                temp_col = next_col
                column_groups[col].append(next_col)
                unique_columns.append(next_col)
                print(f"Merged {col} and {next_col}: common_values = {common_values}")
            else:
                if calc_jump():
                    temp_col = next_col
                    break
                common_values = list(set(df[temp_col].unique()).intersection(set(df[next_col].unique())))
                if len(common_values) >= 1:
                    count += 1
                    temp_col = next_col
                    column_groups[col].append(next_col) 
                    unique_columns.append(next_col) 
                    print(f"Merged {col} and {next_col}: common_values = {common_values}")
                else:break

        unique_columns.append(col)
    
    for group_name, group_columns in column_groups.items():
        if len(group_columns) > 1:
            df[group_name] = df[group_columns].bfill(axis=1).iloc[:, 0]
            print(f"Merged columns {group_columns} into {group_name}")
        else:
            df[group_name] = df[group_columns[0]]
    
    df = df[[col for col in df.columns if col in column_groups]]
    
    return df, count

df_float, count = merge_columns_with_common_values(df_float)

if count == 0:
    print("Nothing changed!")
else:
    print(f"{count} columns were merged.")

In [ ]:
df_int = df_int.astype(float)

df_int = df_int.drop(["client_id","target"],axis=1)
df_int.replace("nan",np.nan, inplace=True)

def merge_columns_with_common_values(df):
    base_toggle = True
    count = 0
    unique_columns = []
    temp_col = "col2"

    column_groups = {}

    for col in df.columns:
        if col in unique_columns: continue

        if col != df.columns[1]: base_toggle = False

        column_groups[col] = [col]

        for next_col in df.columns:
            if col == next_col or next_col in unique_columns:
                continue

            common_values = list(set(df[col].unique()).intersection(set(df[next_col].unique())))
            if np.nan in common_values:
                common_values.remove(np.nan)

            match = re.match(r"([a-zA-Z]+)(\d{1,4})", col)
            match1 = re.match(r"([a-zA-Z]+)(\d{1,4})", next_col)
            def calc_jump():
                if base_toggle:return False
                else:
                    match = re.match(r"([a-zA-Z]+)(\d{1,4})", temp_col)
                    is_jump = (int(match1.group(2)) - int(match.group(2))) > 5
                    return is_jump

            if (len(common_values) >= 1):
                if calc_jump():
                    temp_col = next_col
                    break
                count += 1
                temp_col = next_col
                column_groups[col].append(next_col)
                unique_columns.append(next_col)
                print(f"Merged {col} and {next_col}: common_values = {common_values}")
            else:
                if calc_jump():
                    temp_col = next_col
                    break
                common_values = list(
                    set(df[temp_col].unique()).intersection(set(df[next_col].unique()))
                )
                if len(common_values) >= 1:
                    count += 1
                    temp_col = next_col
                    column_groups[col].append(next_col)
                    unique_columns.append(next_col)
                    print(f"Merged {col} and {next_col}: common_values = {common_values}")
                else:break

        unique_columns.append(col)

    for group_name, group_columns in column_groups.items():
        if len(group_columns) > 1:
            df[group_name] = (df[group_columns].bfill(axis=1).iloc[:, 0])
            print(f"Merged columns {group_columns} into {group_name}")
        else:
            df[group_name] = df[group_columns[0]]

    df = df[[col for col in df.columns if col in column_groups]]

    return df, count

df_int, count = merge_columns_with_common_values(df_int)

if count == 0: print("Nothing changed!")
else: print(f"{count} columns were merged.")

Соединяем обработаные фреймы в один

In [9]:
df_not_sorted = pd.concat([df_object['report_date'],df['client_id'], df['target']],axis=1)
df_object.drop(columns='report_date', inplace=True)

df_not_sorted = pd.concat([df_not_sorted, df_object, df_int, df_float],axis=1)
df_not_sorted.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14456 entries, 0 to 14455
Columns: 168 entries, report_date to col2663
dtypes: Int64(2), float64(129), object(37)
memory usage: 18.6+ MB


Сортируем колонки во фрейме

In [10]:
order_dict = {}
order_list = ['report_date','client_id','target']
columns = df_not_sorted.columns
for column in columns:
    if 'col' in column:
        order_dict[column] = int(re.match(r"([a-zA-Z]+)(\d{1,4})", column).group(2))

sorted_columns = {k: v for k, v in sorted(order_dict.items(), key=lambda item: item[1])}
for k in sorted_columns:
    order_list.append(k)

df = df_not_sorted[order_list]

Удаляем столбцы со ссылками

In [11]:
def has_link(column): return column.astype(str).str.startswith(('http://', 'https://'), na=False).any()

df = df.loc[:, ~df.apply(has_link)]

Переводим report_date в datetime и удаляем все столбцы object, тк оказывается с ними не очень удобно

In [12]:
df['report_date'] = pd.to_datetime(df['report_date'], format='%Y-%m-%d')
df = df.select_dtypes(exclude=['object'])

На этом наша обработка закончена, сохраняем файл

In [13]:
df.to_csv("../processed_train.csv", index=False)